# Model: necks

In [ ]:
#| default_exp model_necks

In [ ]:
#|export
import torch
import torch.nn as nn
from pillarnext_explained.model_utils import BasicBlock, ConvBlock
import torch.utils.checkpoint as cp
import torch.nn.functional as F

## ASPP

In [ ]:
#|exports
class ASPPNeck(nn.Module):
    """
    Atrous Spatial Pyramid Pooling Neck Module.

    This module applies several convolutions with different dilation rates
    to the input feature map and concatenates their outputs. The concatenated
    output is then passed through a convolutional block to produce the final output.
    """
    def __init__(self,
                 in_channels: int # Number of input channels
                 ):

        super(ASPPNeck, self).__init__()

        self.pre_conv = BasicBlock(in_channels)
        self.conv1x1 = nn.Conv2d(
            in_channels, in_channels, kernel_size=1, stride=1, bias=False, padding=0)
        self.weight = nn.Parameter(torch.randn(in_channels, in_channels, 3, 3))
        self.post_conv = ConvBlock(in_channels * 6, in_channels, kernel_size=1, stride=1)

    def _forward(self, x):
        x = self.pre_conv(x)
        branch1x1 = self.conv1x1(x)
        branch1 = F.conv2d(x, self.weight, stride=1,
                           bias=None, padding=1, dilation=1)
        branch6 = F.conv2d(x, self.weight, stride=1,
                           bias=None, padding=6, dilation=6)
        branch12 = F.conv2d(x, self.weight, stride=1,
                            bias=None, padding=12, dilation=12)
        branch18 = F.conv2d(x, self.weight, stride=1,
                            bias=None, padding=18, dilation=18)
        x = self.post_conv(
            torch.cat((x, branch1x1, branch1, branch6, branch12, branch18), dim=1))
        return x

    def forward(self, x):
        if x.requires_grad:
            out = cp.checkpoint(self._forward, x)
        else:
            out = self._forward(x)

        return out

A neural network module which utilizes Atrous Spatial Pyramid Pooling (ASPP). This module is designed to extract and merge features from different receptive fields using convolutions with various dilation rates. It also employs gradient checkpointing to save memory during backpropagation.

In [ ]:
#|eval: false
# Create a sample input tensor with batch size 1, 64 channels, and 128x128 spatial dimensions
input_tensor = torch.randn(1, 64, 128, 128)

# Initialize the ASPPNeck module with 64 input channels
aspp_neck = ASPPNeck(64)

# Pass the input tensor through the ASPPNeck module
output_tensor = aspp_neck(input_tensor)

# Print the shape of the output tensor
print("Output shape:", output_tensor.shape)

Output shape: torch.Size([1, 64, 128, 128])
